# Propulsion

The propulsion subsystem in Aviary organizes and executes models for engine sizing and performance.

Aviary does not natively model gas-turbine performance, and instead relies on user-provided data and/or custom performance models to perform propulsion analysis.

Aviary supports an arbitrary number of propulsor models on a vehicle, each with their own unique properties such as performance characteristics, scaling behaviors, and number of propulsors for that given type. 

```{note}
In Aviary, when we refer to multiple engines of the same type, we say they are homogenous. When you are dealing with multiple engines of different types, we say they are heterogeneous.
``` 

<!-- A diagram would be helpful here, showing X propulsors of type A and Y propulsors of type B on an outline of an aircraft -->
Each unique type of engine is defined by a *engine model*<!-- Link to wherever this class is described - theory guide? -->. In Aviary, an *engine model* contains information on how to size and estimate performance for a single instance of an engine of that type. During analysis, Aviary handles summing performance data to a system level. This way, information is available on the performance of both a single instance of an engine, as well as aircraft-level totals, for other Aviary subsystems to utilize.

Aviary's variable hierarchy makes a distinction between variables that apply to a single instance of an engine, under the `Aircraft.Engine` sub-category, and variables that apply to the propulsion system as a whole under `Aircraft.Propulsion`. As an example, imagine an aircraft with a heterogeneous set of engines (those with unique properties). Maximum sea-level-static thrust would be defined differently per engine vs propulsion system like so:

>*aircraft:engine:scaled_sls_thrust = [1500, 2200], lbf*
>
>*aircraft:engine:num_engines = [3, 2], unitless*
>
>*aircraft:propulsion:total_scaled_sls_thrust = 8900, lbf*

As you can see, variables under *engine* are vectors that store a value for a single each instance of a single type of engine, while variables under *propulsion* only store a single value that is relevant to the entire aircraft rather than any one specific engine or set of engines. Put another way, this aircraft has two hereogenous types of engines, the first that produces 1,500 lbf of thrust *per engine*, with three of those engines on the aircraft, and the second engine that produces 2,200 lbf of thrust *per engine* with only two copies of that engine on the aircraft. The propulsion-level total is 8,900 lbf of thrust, which is every engine physically present on the aircraft combined together (1500\*3 + 2200\*2).

In general, the user does not need to define *propulsion* variables, as Aviary can calculate them given values in *engine*, but they are important to understand when reading results.

In this section, we will explain how to set up the propulsion systems of an Aviary model, focusing on relatively straightforward aircraft configurations.

## Engine Decks
<!--parts of this are probably theory?-->
The primary way Aviary handles engine performance is by interpolating tabular data from a user-defined file that describes performance characteristics for a given engine. Engines modeled in this manner are called *engine decks*<!-- Link to wherever this class is described - theory guide? -->. Engine decks are a type of engine model - they use the same basic interface, but have additional functionality to handle reading and processing data files.

Engine decks can contain a variety of different data to represent any number of propulsion systems, typically ones that utilize combustion such as turbofan or turboshaft engines, including hybridized ones. The most common application of an *engine deck* is to represent a turbofan engine, essentially as black box model that describes the relationship between energy consuption (in the form of fuel) into useful work (the production of thrust).

More advanced applications of *engine decks* are as parts of larger, more detailed *engine models*, such as a turboprop. A turboprop model, as described later, connects a turboshaft engine deck with a gearbox and propeller model to bridge the gap between energy consumption and thrust production, packaged together into a single model that treated as a singular unit by the greater propulsion system.

### Formatting
An *engine deck* data file requires specific formatting for Aviary to correctly interpret. These files must follow the [Aviary data file format](input_files.ipynb). An example of a properly-formatted *engine deck* can be found [here](https://github.com/OpenMDAO/Aviary/blob/main/aviary/models/engines/turbofan_22k.deck).

### Variables

The following engine performance parameters are supported natively by Aviary for use in *engine decks*. If units are not specified, the default units are assumed. A variety of alternate names for these variables are understood by Aviary, but it is recommended to use the official names given here. A column with a header not recognized by Aviary will be ignored, with a warning raised at runtime. This allows for variables not used by Aviary to still be included in a data file, either for reference or compatibility with another analysis tool.

<!-- default variables and their units are not finalized -->
| Variable | Default Units | Required? |
| :--- | :--- | :---: |
| {glue:md}`Mach Number` | unitless | &#x2714; |
| {glue:md}`Altitude` | ft | &#x2714; |
| {glue:md}`Throttle` | unitless | &#x2714; |
| {glue:md}`Hybrid Throttle` | unitless | &#x2718; |
| {glue:md}`Net Thrust` | lbf | &#x2714;* |
| {glue:md}`Gross Thrust` | lbf | &#x2718;* |
| {glue:md}`Ram Drag` | lbf | &#x2718;* |
| {glue:md}`Shaft Power` | hp | &#x2718; |
| {glue:md}`RPM` | rpm | &#x2718; |
| {glue:md}`Fuel Flow Rate` | lbm/h | &#x2718; |
| {glue:md}`Electric Power` | kW | &#x2718; |
| {glue:md}`NOx Rate` | lbm/h | &#x2718; |
| {glue:md}`T4 Temperature` | degR | &#x2718; |

**{glue:md}`Net Thrust` (defined as {glue:md}`Gross Thrust` - {glue:md}`Ram Drag`) is not required if both of those variables are provided for calculation*

{glue:md}`Mach Number`, {glue:md}`Altitude`, and the two throttle parameters are independent variables required to describe the operating conditions of the engine. {glue:md}`Hybrid Throttle` is optional, and is intended for use as a second degree of control for engines using independently controllable fuel- and electric-based power. The remaining variables are dependent on the operating conditions and are therefore typically optional.

*Engine decks* without headers are assumed to contain only the required variable set, in the order specified by the table ({glue:md}`Mach`, {glue:md}`Altitude`, {glue:md}`Throttle`, and {glue:md}`Net Thrust`), and with default units.

Comments may be added to an engine deck data file by using a '`#`' symbol preceding the comment. Anything after this symbol on that line is ignored by Aviary, allowing the user to temporarily remove data points or add in-line comments with context for the data. It is good practice to include comments at the start of the file to explain what kind of engine the data represents, and where it came from.

In [ ]:
# Testing Cell
from aviary.api import Aircraft
from aviary.subsystems.propulsion.engine_deck import (
    aliases,
    default_required_variables,
    dependent_options,
    required_options,
)
from aviary.subsystems.propulsion.utils import EngineModelVariables
from aviary.utils.doctape import check_contains, check_value, get_variable_name, glue_variable
from aviary.variable_info.variable_meta_data import CoreMetaData

vars = [
    'Mach Number',
    'Altitude',
    'Throttle',
    'Hybrid Throttle',
    'Net Thrust',
    'Gross Thrust',
    'Ram Drag',
    'Fuel Flow Rate',
    'Electric Power',
    'NOx Rate',
    'T4 Temperature',
    'Shaft Power',
    'RPM',
]
required = ['Mach', 'Altitude', 'Throttle', 'Net Thrust']

for var in vars:
    glue_variable(var, md_code=True)
for var in required:
    if var not in vars:
        glue_variable(var, md_code=True)


def str_to_enum(list_of_vars):
    for ii, var in enumerate(list_of_vars):
        for enum, alias in aliases.items():
            if var.lower().replace(' ', '_') in alias:
                list_of_vars[ii] = enum
                break
        if not isinstance(list_of_vars[ii], EngineModelVariables):
            raise TypeError(f'"{var}" could not be mapped to EngineModelVariables')


# Check that all of variables in the table have an associated EngineModelVariable
str_to_enum(vars)
str_to_enum(required)
check_value(set(required), default_required_variables)

Aircraft.Engine = Aircraft.Engine
check_contains(required_options, CoreMetaData)
required = (  # f'{Aircraft.Engine.DATA_FILE=}'.split('=')[0],
    get_variable_name(Aircraft.Engine.SCALE_PERFORMANCE),
    get_variable_name(Aircraft.Engine.IGNORE_NEGATIVE_THRUST),
    get_variable_name(Aircraft.Engine.GEOPOTENTIAL_ALT),
    get_variable_name(Aircraft.Engine.GENERATE_FLIGHT_IDLE),
)
required_options_list = ''
for var in required:
    required_options_list += f'* `{var}`\n'
required_options_list += (
    f'* `{get_variable_name(Aircraft.Engine.NUM_WING_ENGINES)}` and/or '
    + f'`{get_variable_name(Aircraft.Engine.NUM_FUSELAGE_ENGINES)}`\n'
)
glue_variable('required_options', required_options_list, display=True)

GENERATE_FLIGHT_IDLE = (
    Aircraft.Engine.FLIGHT_IDLE_THRUST_FRACTION,
    Aircraft.Engine.FLIGHT_IDLE_MIN_FRACTION,
    Aircraft.Engine.FLIGHT_IDLE_MAX_FRACTION,
)
check_value(dependent_options[Aircraft.Engine.GENERATE_FLIGHT_IDLE], GENERATE_FLIGHT_IDLE)
flight_idle_options = (
    f'* `{get_variable_name(Aircraft.Engine.FLIGHT_IDLE_THRUST_FRACTION)}`\n'
    + f'* `{get_variable_name(Aircraft.Engine.FLIGHT_IDLE_MIN_FRACTION)}`\n'
    + f'* `{get_variable_name(Aircraft.Engine.FLIGHT_IDLE_MAX_FRACTION)}`\n'
)
glue_variable('flight_idle_options', flight_idle_options, display=True)

## Setting Up Propulsion Analysis

### Beginner Guide

To add an engine deck to Aviary, the minimum set of variables to describe it must be provided in your input file. This list includes:
<!-- This list is for EngineDecks specifically, does that need clarification? No other engine model is currently planned to be supported with a level1 interface right? -->
```{glue:md} required_options
:format: myst
```

<!--At a system level, there a few variables you must include that affect all engines. This list includes: -->

<!-- Additional variables are required but will depend on preprocessor behavior, which is not finalized. NUM_WING_ENGINES & NUM_FUSELAGE_ENGINES vs NUM_ENGINES behavior, SCALE_FACTOR vs. SCALED_SLS_THRUST, etc. -->
If generating flight idle points is desired, the following variables are also required. More information on flight idle generation is available here <!--TODO link here-->.

```{glue:md} flight_idle_options
:format: myst
```

If you are missing any required variables, you may see a warning at runtime, depending on verbosity level. Aviary will try using the default value for the missing variable, which may affect analysis results.

```
UserWarning: <aircraft:engine:scale_performance> is a required option for EngineDecks, but has not been specified for EngineDeck <example>. The default value will be used.
```

<!-- See !!!LINK HERE!!! for a complete list of all available options to define engine behavior -->

<!-- Section on setting up Propulsion-level variables, which ones are required? -->

In [ ]:
# Testing Cell
from aviary.api import Aircraft
from aviary.subsystems.propulsion.engine_deck import dependent_options
from aviary.utils.doctape import check_value

ae = Aircraft.Engine

required = (
    ae.FLIGHT_IDLE_THRUST_FRACTION,
    ae.FLIGHT_IDLE_MIN_FRACTION,
    ae.FLIGHT_IDLE_MAX_FRACTION,
)
check_value(required, dependent_options[ae.GENERATE_FLIGHT_IDLE])

### Intermediate Guide

Engine models are defined in Aviary using an {glue:md}`EngineModel` object. An {glue:md}`EngineModel` is responsible for handling many tasks required to prepare an engine for use in Aviary, such as reading engine data from a file in the case of an {glue:md}`EngineDeck` (which is a child class of {glue:md}`EngineModel`). <!-- link to theory guide? -->

An {glue:md}`EngineModel` (and classes inheriting it) can be manually created and added to the Aviary problem, instead of being created from the values provided via an Aviary input file. This is extremely useful when setting up an aircraft with multiple heterogenous types of engines, each with unique properties, or using a custom engine model. An {glue:md}`EngineModel` requires an {glue:md}`AviaryValues` object containing the variables required for that engine (such as those outlined in the Beginner Guide example for {glue:md}`EngineDeck`)<!-- link to that subsection -->.

In [ ]:
# Testing Cell
import inspect

from aviary.subsystems.propulsion.engine_deck import EngineDeck
from aviary.subsystems.propulsion.engine_model import EngineModel
from aviary.utils.aviary_values import AviaryValues
from aviary.utils.doctape import check_value, get_variable_name, glue_variable

check_value(EngineDeck.__base__, EngineModel)
type_hint = inspect.signature(EngineModel).parameters['options'].annotation
check_value(type_hint, AviaryValues)

glue_variable(EngineModel.__name__, md_code=True)
glue_variable(EngineDeck.__name__, md_code=True)
glue_variable(AviaryValues.__name__, md_code=True)
glue_variable(get_variable_name(Aircraft.Design.DRAG_POLAR), md_code=True)

In [ ]:
import aviary.api as av
from aviary.api import Aircraft

aviary_options = av.AviaryValues()

# Add relevant inputs and options to engine_options
# using engine_options.set_val(...)
filename = av.get_path('models/engines/turbofan_28k.deck')
aviary_options.set_val(Aircraft.Engine.DATA_FILE, filename)
# example of setting an engine variable to a specific value
aviary_options.set_val(Aircraft.Engine.SCALE_FACTOR, 0.9)

# It is assumed here that aviary_options is configured to have
# all inputs needed for analysis, except engine-level values.
# Because not all required options were defined in this example, we
# see a number of warnings about the missing variables.
engine_deck = av.EngineDeck(name='example', options=aviary_options)

In this example code snippet, we didn't set a value for every required variable and option previously listed - because of this, we get a number of warnings letting us know which ones we forgot to set, and that the default values specified will be used.

Once an {glue:md}`EngineDeck` has been successfully created, it must be attached to the vehicle model you want to analyze. The simplest way to do this is to take advantage of the propulsion preprocessor utility. This preprocessor handles all of the details of getting data related to `EngineModels`, which may change during initialization, correctly set up in the `AviaryValues` object which is used to define the vehicle at the Aviary problem level. This is a critical step, because once Aviary begins analysis it assumes that all of the relevant data present in individual `EngineModels` have made their way into the vehicle-level data.

In [ ]:
av.preprocess_propulsion(aviary_options=aviary_options, engine_models=[engine_deck])

#### Heterogenous Engine Sets
In this example, our aircraft model has two distinct types of engines, which are assumed to have different properties. To keep this example simple, we will build an aircraft with two sets of engines based on the `EngineDeck` we just set up, but one set of engines is scaled to half size using `Aircraft.Engine.SCALE_FACTOR`.

In [ ]:
import aviary.api as av
from aviary.utils.doctape import get_variable_name, glue_variable

filename = 'models/engines/turbofan_28k.deck'
options = av.AviaryValues()
options.set_val(av.Aircraft.Engine.DATA_FILE, av.get_path(filename))

engine_deck_1 = av.EngineModel(options=options)

# now let's change the second engine to be scaled smaller by changing options
options.set_val(Aircraft.Engine.SCALE_FACTOR, 0.5)
engine_deck_2 = av.EngineModel(options=options)

*aviary_options* is modified in-place with updated values to represent our new *engine_deck*. When working with multiple heterogenous engines, simply provide `preprocess_propulsion()` with a list of all `EngineModels`, like so:

In [ ]:
# Test Cell
aviary_options = av.AviaryValues()
# set this to avoid warning in later cell
aviary_options.set_val(Aircraft.Engine.NUM_WING_ENGINES, 2)

In [ ]:
av.preprocess_propulsion(
    aviary_options=aviary_options, engine_models=[engine_deck_1, engine_deck_2]
)

In [ ]:
# Testing Cell
print(f'Interpolation Method: {aviary_options.get_val(Aircraft.Engine.SCALE_FACTOR)}')

Note how calling `preprocess_propulsion()` changes `Aircraft.Engine.SCALE_FACTOR`, which was individually defined for each `EngineDeck`, into a list containing both values. This way Aviary can access information on every `EngineModel` present on the aircraft without needing access to each individual `EngineModel` object. This is important to be aware of if you are manually setting any variables related to engines or propulsion rather than using the built-in methods and functions presented in the Intermediate guide, which will do this work for you.

### Advanced Guide

This section is a work in progress. Please check back later for more information.